In [11]:
from selenium import webdriver
driver = webdriver.Chrome()

### API - requests

In [2]:
# 서버에 request 요청하고 response 응답을 받는 패키지
import requests

#### 네이버 주식 데이터 크롤링
- api 사용: json 파싱해서 데이터를 크롤링 합니다.
- 네이버 주식 페이지에서 주식 데이터를 가져와서 pandas DataFrame으로 변환
- 네이버 주식: http://m.stock.naver.com/sise/siseIndex.nhn
- 국내 주식: http://m.stock.naver.com/api/json/sise/siseListJson.nhn?menu=market_sum&sosok=0

In [5]:
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
    menu=market_sum&sosok=0&pageSize="+str(pageSize)+"&page=1"+str(page)
url = make_url()
url

'http://m.stock.naver.com/api/json/sise/siseListJson.nhn?    menu=market_sum&sosok=0&pageSize=10&page=11'

In [6]:
import pandas as pd

In [7]:
def get_data(url):
    """
    URL을 받으면 json으로 데이터를 가져와서 pandas로 만들어주는 함수
    """
    response = requests.get(url)
    json_info = response.json()
    companies = json_info["result"]["itemList"]
    df = pd.DataFrame(columns=["종목","시세","전일비","등락율","시가총액","거래량"])
    for company in companies:
        df.loc[len(df)] = {
            "종목":company["nm"],
            "시세":company["nv"],
            "전일비":company["cv"],
            "등락율":company["cr"],
            "시가총액":company["mks"],
            "거래량":company["aq"],
        }
    return df
#     print(company)
#     return response.text
#     return json_info
get_data(url)

,종목,시세,전일비,등락율,시가총액,거래량
0,동서,25900,-300,-1.15,25822,7353
1,현대백화점,110500,-500,-0.45,25860,2606
2,포스코대우,20900,-150,-0.71,25785,15247
3,한샘,109000,-1000,-0.91,25652,1247
4,OCI,105500,-1000,-0.94,25161,17064
5,LS,78000,-400,-0.51,25116,4141
6,대우건설,6020,-40,-0.66,25020,54285
7,한화,33400,150,0.45,25036,8242
8,HDC현대산업개발,56300,-600,-1.05,24737,28280
9,녹십자,210500,-3000,-1.41,24600,1738


In [8]:
url = make_url(100,2)
df = get_data(url)
df.tail()

,종목,시세,전일비,등락율,시가총액,거래량
95,KBSTAR 헬스케어,13005,0,0.00,124,0
96,TIGER 대형성장,9125,0,0.00,123,0
97,미래에셋 인버스 원유선물혼합 ETN(H),12280,0,0.00,123,0
98,KINDEX 코스닥(합성),12275,0,0.00,123,0
99,KODEX 턴어라운드투자,12255,-5,-0.04,123,1


In [9]:
from pandas.io.json import json_normalize

In [10]:
def get_data2(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    return json_normalize(companys)

url = make_url(100,2)
df = get_data2(url)
df.tail()

,aa,aq,cd,cr,cv,mks,mt,nm,nv,pcv,rf
95,0,0,292100,0.00,0,123,0,TIGER 대형성장,9125,9125,3
96,240,21079,000225,-6.06,-700,123,0,유유제약1우,10850,11550,5
97,0,0,520011,0.00,0,123,0,미래에셋 인버스 원유선물혼합 ETN(H),12280,12280,3
98,0,0,251890,0.00,0,123,0,KINDEX 코스닥(합성),12275,12275,3
99,0,2,260200,-0.62,-95,123,0,KOSEF 배당바이백Plus,15325,15420,5


### dark sky api
- 날씨 정보를 알려주는 api
- https://darksky.net/dev

In [29]:
FORECAST_TOKEN = "8e7c9846ca596c4aa29c96f0cfb0642c"

In [39]:
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}".format(FORECAST_TOKEN,lat,lng)
    response = requests.get(url)
    json_info = response.json()
    print(url)
    return json_info["timezone"],json_info["hourly"]["summary"]

In [40]:
lat = 37.512
lng = 126.954
forecast(lat, lng)

https://api.darksky.net/forecast/8e7c9846ca596c4aa29c96f0cfb0642c/37.512,126.954


('Asia/Seoul', 'Mostly cloudy throughout the day.')

In [8]:
# dark sky api 날씨 정보를 가지고 오는 패키지
import forecastio

In [50]:
# forecastio 패키지 사용
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHour = forecast.hourly()
    return byHour.summary

In [51]:
lat = 37.512
lng = 126.954
forecast2(lat, lng)

'Mostly cloudy throughout the day.'

### BeautifulSoup

- 문자열로 되어 있는 html 코드를 css selector 나 xpath를 이용해서 특정 엘리먼트의 데이터를 쉽게 가져올 수 있도록 하는 패키지

In [13]:
# bs4
from bs4 import BeautifulSoup

In [52]:
# 네이버 검색어 순위 데이터 크롤링

In [14]:
def naver_top20():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("https://www.naver.com")
#     print(response.content)
#     print(response.text)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select('.ah_roll_area > .ah_l > .ah_item') # 여기에 css selector를 넣어준다.
#     print(len(keywords))
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
        }
    return df
    
naver_top20()

,rank,keyword
0,1,최율
1,2,스페인 이란
2,3,하지
3,4,우루과이 사우디
4,5,한끼줍쇼
5,6,박주호
6,7,덴마크 호주
7,8,lg디스플레이
8,9,나도 엄마야 등장인물
9,10,남산외인아파트


In [84]:
import time
ls = []
for idx in range(2):
    print(idx)
    df = naver_top20()
    ls.append(df)
    time.sleep(60*5)

0
1


In [85]:
ls[0]

,rank,keyword
0,1,러시아 사우디 하이라이트
1,2,이희준
2,3,이집트 우루과이
3,4,박희순
4,5,이혜정
5,6,박예진
6,7,프로듀스48
7,8,살라
8,9,스페인 포르투갈
9,10,장윤정 엄마


In [86]:
ls[1]

,rank,keyword
0,1,러시아 사우디 하이라이트
1,2,이희준
2,3,이집트 우루과이
3,4,박예진
4,5,김환희
5,6,이혜정
6,7,박희순
7,8,장윤정
8,9,프로듀스48
9,10,살라


In [15]:
def naver_top20():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("https://www.naver.com")
#     print(response.content)
#     print(response.text)
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select('.ah_roll_area > .ah_l > .ah_item') # 여기에 css selector를 넣어준다.
#     print(len(keyword))
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
        }
    return df
    
naver_top20()

,rank,keyword
0,1,최율
1,2,스페인 이란
2,3,하지
3,4,우루과이 사우디
4,5,한끼줍쇼
5,6,박주호
6,7,덴마크 호주
7,8,lg디스플레이
8,9,나도 엄마야 등장인물
9,10,남산외인아파트


In [16]:
# 문제. 다음 검색어 순위를 가져와서, 네이버 검색어 순위와 중복되는 키워드만 출력

def daum_top10():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("https://www.daum.net")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select('.realtime_part > .list_hotissue.issue_row > li')
#     print(keywords)
#     print(len(keywords))
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ir_wa").text,
            "keyword":keyword.select_one(".link_issue").text,
        }
    return df

daum_top10()

,rank,keyword
0,1위,남산 외인아파트
1,2위,하지
2,3위,조재현
3,4위,재일교포 여배우
4,5위,김제동
5,6위,김종태
6,7위,알렉스
7,8위,진성준
8,9위,모로코
9,10위,미륵사지 석탑


In [17]:
naver_df = naver_top20()
daum_df = daum_top10()
naver_df

,rank,keyword
0,1,최율
1,2,스페인 이란
2,3,하지
3,4,우루과이 사우디
4,5,한끼줍쇼
5,6,박주호
6,7,덴마크 호주
7,8,lg디스플레이
8,9,나도 엄마야 등장인물
9,10,남산외인아파트


In [18]:
daum_df

,rank,keyword
0,1위,남산 외인아파트
1,2위,하지
2,3위,조재현
3,4위,재일교포 여배우
4,5위,김제동
5,6위,김종태
6,7위,알렉스
7,8위,진성준
8,9위,모로코
9,10위,미륵사지 석탑


In [28]:
# 네이버, 다음 중복된 실시간 검색어 찾아 비교 1
result = [
        keyword 
        for keyword in daum_df["keyword"] 
          if naver_df["keyword"].str.contains(keyword).any()
]
result

['하지', '알렉스']

In [27]:
# 네이버, 다음 중복된 실시간 검색어 찾아 비교 2
set(daum_df["keyword"]) & set(naver_df["keyword"])

{'알렉스', '하지'}

In [156]:
# 중복된 키워드 찾아 비교
daum_df["keyword"], naver_df["keyword"]
result = 

(0       국가장학금
 1    월드컵 경기일정
 2         유인태
 3      한국장학재단
 4         조윤선
 5         정우택
 6         최순실
 7         김환희
 8         임창호
 9     러시아 사우디
 Name: keyword, dtype: object, 0     러시아 사우디 하이라이트
 1               박희순
 2          이집트 우루과이
 3               박예진
 4             포항 약국
 5                살라
 6               김우리
 7               안희정
 8            장윤정 엄마
 9               이집트
 10               썰전
 11              러시아
 12          만물상깻잎조림
 13            김우리 딸
 14          로비 윌리엄스
 15            김우리나이
 16            투머치 뜻
 17          러시아 사우디
 18              박지성
 19           모로코 이란
 Name: keyword, dtype: object)

### file download

In [157]:
# url 링크로 파일 다운로드 받는 방법

In [158]:
title = "apple.mp4"
download_url = "https://bit.ly/2t8UAaw"
len(download_url)

22

In [162]:
def download(title, download_url):
    response = requests.get(download_url, stream=True)
    path = "./" + title
    size=0
    with open(path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                size += 1024
                f.write(chunk)
    return size

In [164]:
size = download(title, download_url)

In [165]:
size

4497408

In [171]:
round(size/1024/1024,3)

4.289